Use transformers? GPT api,  
How to process data  

divide in sentences, lowercase, firstword only (if not detected as NER)  
then all to tokens


**Preprocessing Steps for identification:**  
- <u>Lowercasing:</u>  
Convert the text to lowercase to ensure consistency and avoid duplicating words with different cases. For example, "Movie" and "movie" should be treated as the same word.
  
- <u>Tokenization:  </u>  
Split the text into individual words or tokens. Tokenization helps to break down the text into meaningful units for further processing. You can use libraries like NLTK or spaCy for tokenization.
  
- <u>Removal of Special Characters and Punctuation:</u>  
Remove special characters, punctuation marks, and symbols from the text. These elements generally do not contribute significantly to the semantic meaning and can introduce noise in the analysis.
  
- <u>Removal of Stopwords:</u>    
Eliminate common words, known as stopwords (e.g., "the," "is," "and"), as they typically do not carry significant information for identification or similarity tasks. NLTK provides a list of stopwords for various languages.
  
- <u>Lemmatization or Stemming:</u>  
Reduce words to their base or root form to unify semantically similar words. This step helps to handle variations such as singular/plural forms and verb tenses. NLTK provides modules for lemmatization and stemming.
  
- <u>Removal of Numeric Tokens:</u>  
Exclude numeric tokens or words from the preprocessing steps, as they may not contribute much to the identification or similarity analysis unless specific numerical information is essential in your task.





In [2]:
import pandas as pd
import nltk
import spacy 

/Users/ivan/opt/anaconda3/envs/sem2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-06-26 20:20:15.508618: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# python -m spacy download en_core_web_sm
nlp = spacy.load("en_core_web_lg")
# import en_core_web_sm
# nlp = en_core_web_sm.load()
doc = nlp("This is a sentence.")
print([(w.text, w.pos_) for w in doc])

[('This', 'PRON'), ('is', 'AUX'), ('a', 'DET'), ('sentence', 'NOUN'), ('.', 'PUNCT')]


In [14]:
df_raw=pd.read_csv('Movie-2023-06-15.csv')
print(len(df_raw))


89


In [5]:
print(df_raw.columns)

df=df_raw[['id', 'imdb_id', 'name', 'year','starts_name', 'description']]
print(len(df))
print(df.columns)
df.head()

Index(['id', 'imdb_id', 'name', 'year', 'rating', 'certificate', 'duration',
       'genre', 'votes', 'gross_income', 'director_id', 'director_name',
       'starts_id', 'starts_name', 'description', 'url', 'payload', 'status',
       'attempt', 'created', 'updated'],
      dtype='object')
89
Index(['id', 'imdb_id', 'name', 'year', 'starts_name', 'description'], dtype='object')


,id,imdb_id,name,year,starts_name,description
0,c8de01ac-04c2-444a-9d9b-26cb213a9b2f,tt0241527,Harry Potter and the Sorcerer's Stone,2001,"Richard Harris,Maggie Smith,Robbie Coltrane,Sa...",An orphaned boy enrolls in a school of wizardr...
1,d76d12d5-caa4-4101-93da-52012300efa6,tt0117060,Mission: Impossible,1996,"Tom Cruise,Jon Voight,Emmanuelle Bart,Henry Cz...","An American agent, under false suspicion of di..."
2,b6e07841-25c9-49b8-b563-ef3007753017,tt0068646,The Godfather,1972,"Marlon Brando,Al Pacino,James Caan,Richard S. ...","Don Vito Corleone, head of a mafia family, dec..."
3,9b175434-f80c-4d7a-a98d-e0551ba61af3,tt1201607,Harry Potter and the Deathly Hallows: Part 2,2011,"Ralph Fiennes,Michael Gambon,Alan Rickman,Dani...","Harry, Ron, and Hermione search for Voldemort'..."
4,ff11b3f0-c8cb-426b-a51b-0790bedbff8b,tt0090605,Aliens,1986,"Sigourney Weaver,Carrie Henn,Michael Biehn,Pau...","Decades after surviving the Nostromo incident,..."


In [13]:
print(df.loc[0,'name'])

Harry Potter and the Sorcerer's Stone


In [17]:
summary_raw_hp2 = "In this movie Ginny open it with a diary, and there is a big snake. Emma Watson as hermione, ron help Harry to defend from Voldemort, also Harry can talk to it in the girls bathroom. "
summary_raw_hp8 = "In this movie Harry have destroyed most Horcruxes, and he Hermione and Ron Have to fight Voldemort. There is a huge battle of wizards in the castle of Hogwarts. Weasly die in the fight, but one twin lose his ear. "
summary_raw_gf='a gangster try to get into a family business,Corleone with the italians. they are in the mafia family, the head of the Corleone Mafia family '
summary_raw_mi=" the IMF agent Ethan Hunt gets bretrayed and have to steal the noc list."
dataset_raw = df['description'].tolist()


# NLTK approach1

In [21]:
from nltk.tokenize import sent_tokenize, word_tokenize

from nltk.corpus import stopwords
stopword = stopwords.words('english')

def clean_text(text):
    text = text.replace('\n','. ')#.replace('','').replace('','').replace('','')
#     text = text.replace(',',' ').replace('!','').replace('?','')
    return text

def remove_punct(text, lower=False):
    if(type(text)==str):
        text = text.replace('\n','. ')#.replace('','').replace('','').replace('','')
        text = text.replace(',',' ').replace('!','').replace('?',' ')
        text = text.replace('.',' ').replace('#','').replace('$',' ')
        text = text.replace('^',' ').replace('&','and').replace(';',' ')
        text = text.replace('  ',' ')
        return text
    elif(type(text)==list):
        _words=[]
        for w in text:
            if w.isalnum():
                _words.append(w if not lower else w.lower())
#         print('remove_punct',len(_words))
        return _words

def get_sentences(text):
    return sent_tokenize(text)

def get_tokens(text):
    words =[]
    for w in word_tokenize(text):
        if w.isalnum():
            words.append(w)
    return words
    

def remove_stopwords(words):
    just_words=[]
    for word in words:
        if word.lower() not in stopword:
            just_words.append(word)
#     print('just_words',len(just_words))
    return just_words
    
    
def lemmatize(text):
    record_lemmatized = [lemmatizer.lemmatize(token) for token in text]
    return record_lemmatized
    
    
def list_to_string(ls):
    return " ".join(ls)
    
# clean words: boooook
# Remove non stop words of 2 letters






# Cleaning approach 1:
    # tokenize everything, remove stop words, lower all, remove punctutation

# set tokens each description
dataset_tokens= [ get_tokens(x) for x in dataset_raw] 

# remove stopwords
dataset_nostopwords= [remove_stopwords(x) for x in dataset_tokens]

# remove punctuation
dataset_main_words= [remove_punct(x, lower=False) for x in dataset_nostopwords ]

# Lower all
# dataset_main_words_lower=[x.lower() for x in dataset_main_words]

# Lemmatize words
# record_lemmatized = [lemmatizer.lemmatize(token) for token in dataset_main_words]


# Back to string
dataset_clean_text = [ list_to_string(x) for x in dataset_main_words]

def preprocess_text_ap1(x):
       # tokenize everything, remove stop words, lower all, remove punctutation
    # set tokens each description
    dataset_tokens= get_tokens(x)
    # remove stopwords
    dataset_nostopwords= remove_stopwords(dataset_tokens)
    # remove punctuation
    dataset_main_words= remove_punct(dataset_nostopwords, lower=False)
    # Lower all
    # dataset_main_words_lower=[x.lower() for x in dataset_main_words]
    # Back to string
    dataset_clean_text =  list_to_string(dataset_main_words)
    return dataset_clean_text

# SPACY

Do not lower all words, lower, Only, after found is not a NER Name person or company or movie:  
i.e. Ethan Hunt - > hunt is now vverb, not last name  
Lily Potter -> is now lily (flower) potter (proffesion)  
Max Power -> all power

In [22]:
#Datasets
## Ment to do once

# dataset=dataset_raw
# datasets=[" ".join(get_tokens(clean_text(x))) for x in dataset_raw]

dataset_clean_text=[preprocess_text_ap1(x) for x in dataset_clean_text]

datasets=dataset_clean_text
dataset_docs = [nlp(record) for record in datasets]


In [23]:
datasets= dataset_docs.copy()

In [24]:
summary_raw=summary_raw_hp8 #->hp4 -> cambio
# summary_raw=summary_raw_gf #->Han Solo Jabba Hutt Rebels
# summary_raw=summary_raw_mi #->widow Emily French Norma Varden
# summary=" ".join(get_tokens(preprocess_text_ap1(summary_raw)))
summary=(preprocess_text_ap1(summary_raw))


summary_doc = nlp(summary)

similarity_scores = [summary_doc.similarity(doc) for doc in dataset_docs]


most_similar_index = similarity_scores.index(max(similarity_scores))
most_similar_record = datasets[most_similar_index]

# print("MySummary:",summary_raw,'\nSimilar:')
print(f"\tMysUmmary:\n {summary_raw},\n\tSummary NLP:\n{summary}\nSimilar:\n")

most_similar_record[:100]

	MysUmmary:
 In this movie Harry have destroyed most Horcruxes, and he Hermione and Ron Have to fight Voldemort. There is a huge battle of wizards in the castle of Hogwarts. Weasly die in the fight, but one twin lose his ear. ,
	Summary NLP:
movie Harry destroyed Horcruxes Hermione Ron fight Voldemort huge battle wizards castle Hogwarts Weasly die fight one twin lose ear
Similar:



Harry Ron Hermione search Voldemort remaining Horcruxes effort destroy Dark Lord final battle rages Hogwarts burying Dobby garden Shell cottage Harry Potter Daniel Radcliffe Warwick Davis help get Lestrange vault Gringotts Voldemort Horcruxes exchange Godric Gryffindor Ollivander John Hurt Wandmaker warns Harry wo stand Voldemort Ralph Fiennes Elder Wand arrived Gringotts Hermione Emma Watson disguised Bellatrix Helena Bonham Carter using Polyjuice Potion Ron Rupert Grint disguised random wizard Harry Griphook go Invisibility Cloak help Imperius curse manage get carts take vaults cover blown Gringotts security attacks manage get Lestrange vault find Horcrux Helga Hufflepuff Cup Griphook betrays flees sword yelling

In [25]:
# datasets_results_df = pd.DataFrame(columns=['title','description','similarity'], data=[range(0,len(similarity_scores)),datasets,similarity_scores])
# datasets_results_df = pd.DataFrame(columns=['title'], data=df['title'])
datasets_results_df=df.copy()
datasets_results_df['description_nlp']=datasets
datasets_results_df['similarity']=similarity_scores
datasets_results_df.sort_values(by='similarity', inplace=True, ascending=False)
datasets_results_df.head()

,id,imdb_id,name,year,starts_name,description,description_nlp,similarity
3,9b175434-f80c-4d7a-a98d-e0551ba61af3,tt1201607,Harry Potter and the Deathly Hallows: Part 2,2011,"Ralph Fiennes,Michael Gambon,Alan Rickman,Dani...","Harry, Ron, and Hermione search for Voldemort'...","(Harry, Ron, Hermione, search, Voldemort, rema...",0.816946
7,fa5f656d-54b3-4095-bf77-9a8c39443b3a,tt0167260,The Lord of the Rings: The Return of the King,2003,"Noel Appleby,Ali Astin,Sean Astin,David Aston,...",Gandalf and Aragorn lead the World of Men agai...,"(Gandalf, Aragorn, lead, World, Men, Sauron, a...",0.771519
77,19ebeaef-021e-4e3e-9dc7-4d4d319d2fff,tt0167261,The Lord of the Rings: The Two Towers,2002,"Bruce Allpress,Sean Astin,John Bach,Sala Baker...",While Frodo and Sam edge closer to Mordor with...,"(Frodo, Sam, edge, closer, Mordor, help, shift...",0.765762
0,c8de01ac-04c2-444a-9d9b-26cb213a9b2f,tt0241527,Harry Potter and the Sorcerer's Stone,2001,"Richard Harris,Maggie Smith,Robbie Coltrane,Sa...",An orphaned boy enrolls in a school of wizardr...,"(orphaned, boy, enrolls, school, wizardry, lea...",0.764040
41,911331de-14f6-43d5-a221-c0b821e4c8c7,tt0120737,The Lord of the Rings: The Fellowship of the Ring,2001,"Alan Howard,Noel Appleby,Sean Astin,Sala Baker...",A meek Hobbit from the Shire and eight compani...,"(meek, Hobbit, Shire, eight, companions, set, ...",0.732214


# Probando el accuracy


In [28]:
df_jw = pd.read_csv('Movie-2023-06-15_justwatch.csv')
del df_jw['Unnamed: 0']

In [26]:
def find_similar_movie(_synopsis):
    summary_raw=_synopsis 
    summary=(preprocess_text_ap1(summary_raw))
    summary_doc = nlp(summary)
    similarity_scores = [summary_doc.similarity(doc) for doc in dataset_docs]
    most_similar_index = similarity_scores.index(max(similarity_scores))
  
    df_match = df.iloc[most_similar_index]#[df['']]
    # THe best this, is to find the movie in the dataset, return title, year and link
    return df_match #most_similar_record.text


In [32]:
df_jw

,id,imdb_id,description_clean,name,year,starts_name,description,justwatch_name,justwatch_snps
0,c8de01ac-04c2-444a-9d9b-26cb213a9b2f,tt0241527,orphaned boy enrolls school wizardry learns tr...,Harry Potter and the Sorcerer's Stone,2001,"Richard Harris,Maggie Smith,Robbie Coltrane,Sa...",An orphaned boy enrolls in a school of wizardr...,harry-potter-and-the-philosophers-stone,Harry Potter has lived under the stairs at his...
1,d76d12d5-caa4-4101-93da-52012300efa6,tt0117060,American agent false suspicion disloyalty must...,Mission: Impossible,1996,"Tom Cruise,Jon Voight,Emmanuelle Bart,Henry Cz...","An American agent, under false suspicion of di...",mission-impossible,"When Ethan Hunt, the leader of a crack espiona..."
2,b6e07841-25c9-49b8-b563-ef3007753017,tt0068646,Vito Corleone head mafia family decides hand e...,The Godfather,1972,"Marlon Brando,Al Pacino,James Caan,Richard S. ...","Don Vito Corleone, head of a mafia family, dec...",the-godfather,"Spanning the years 1945 to 1955, a chronicle o..."
3,9b175434-f80c-4d7a-a98d-e0551ba61af3,tt1201607,Harry Ron Hermione search Voldemort remaining ...,Harry Potter and the Deathly Hallows: Part 2,2011,"Ralph Fiennes,Michael Gambon,Alan Rickman,Dani...","Harry, Ron, and Hermione search for Voldemort'...",harry-potter-and-the-deathly-hallows-part-2,"Harry, Ron and Hermione continue their quest t..."
4,ff11b3f0-c8cb-426b-a51b-0790bedbff8b,tt0090605,Decades surviving Nostromo incident Ellen Ripl...,Aliens,1986,"Sigourney Weaver,Carrie Henn,Michael Biehn,Pau...","Decades after surviving the Nostromo incident,...",aliens,Two young boys discuss their favorite movies a...
...,...,...,...,...,...,...,...,...,...
84,0cf96760-1af3-463a-9be3-782356685573,tt0056172,story Lawrence English officer successfully un...,Lawrence of Arabia,1962,"Peter O'Toole,Alec Guinness,Anthony Quinn,Jack...","The story of T.E. Lawrence, the English office...",lawrence-of-arabia,The story of British officer T.E. Lawrence's m...
85,04b3a719-1b70-462a-b7c5-3fae85109e4b,tt0050825,refusing attack enemy position general accuses...,Paths of Glory,1957,"Kirk Douglas,Ralph Meeker,Adolphe Menjou,Georg...","After refusing to attack an enemy position, a ...",paths-of-glory,A commanding officer defends three scapegoats ...
86,02311676-0ca3-4149-a40e-4346f0b910da,tt0364569,kidnapped imprisoned fifteen years Oh released...,Oldboy,2003,"Choi Min-sik,Yoo Ji-tae,Kang Hye-jeong,Kim Bye...",After being kidnapped and imprisoned for fifte...,oldboy,An everyday man has only three and a half days...
87,01e562bc-9e59-4019-806d-97deb6a82f2b,tt7286456,rise Arthur Fleck aspiring comedian pariah Got...,Joker,2019,"Joaquin Phoenix,Robert De Niro,Zazie Beetz,Fra...","The rise of Arthur Fleck, from aspiring stand-...",joker,"Mannar Mannan, an ordinary villager, decides t..."


In [45]:
matches=0
fails=0
for i,  name in df_jw.iterrows():
    if(type(name['justwatch_snps'])!=str):
        print("\n\n>>>>ERROR ON ",name['justwatch_name'],name['justwatch_snps'])
        fails=fails+1
        continue
    _summary=name['justwatch_snps']
    _match=find_similar_movie(_summary)
    if(name['name'] == _match['name']):
        matches=matches+1
        print("✅", end="")
    else:
        print("❌", end="")
    print("Sent:",name['name'] ,"\tPredicted:", _match['name'])
print("\n\nTotal found ",matches,'of',len(df_jw)-fails , (matches*100/(len(df_jw)-fails)), "%")

✅Sent: Harry Potter and the Sorcerer's Stone 	Predicted: Harry Potter and the Sorcerer's Stone
✅Sent: Mission: Impossible 	Predicted: Mission: Impossible
✅Sent: The Godfather 	Predicted: The Godfather
❌Sent: Harry Potter and the Deathly Hallows: Part 2 	Predicted: Harry Potter and the Sorcerer's Stone
❌Sent: Aliens 	Predicted: Singin' in the Rain
✅Sent: Apocalypse Now 	Predicted: Apocalypse Now
✅Sent: Avengers: Endgame 	Predicted: Avengers: Endgame
❌Sent: The Lord of the Rings: The Return of the King 	Predicted: The Lord of the Rings: The Fellowship of the Ring


>>>>ERROR ON  sunset-blvd nan
✅Sent: Django Unchained 	Predicted: Django Unchained
✅Sent: The Usual Suspects 	Predicted: The Usual Suspects
❌Sent: Rear Window 	Predicted: M
❌Sent: The Good, the Bad and the Ugly 	Predicted: Come and See
❌Sent: Pulp Fiction 	Predicted: Once Upon a Time in the West
❌Sent: Whiplash 	Predicted: The Lives of Others
❌Sent: 12 Angry Men 	Predicted: Witness for the Prosecution
✅Sent: Dr. Strangelove or

#  
# =============================  
# Approach2
# =============================  


# Spacy
Tokenize and process your summary and dataset records using spaCy.  
Extract features or embeddings from the text, such as word vectors.  
Calculate the similarity between the summary and each dataset record using cosine similarity or other distance metrics.  
Sort the records based on their similarity scores to find the most similar one.

In [ ]:
ss=nlp("Sir Ethan Hunt is a member of MI6")
ss

In [ ]:
import spacy
nlp = spacy.load("en_core_web_lg")
#python -m spacy download en_core_web_lg



In [ ]:
summary

In [ ]:
similarity_scores[:10]

In [ ]:
datasets

In [ ]:
dataset = dataset_raw
dataset_docs = [nlp(record) for record in dataset]


In [ ]:
type(similarity_scores)


In [ ]:
summary = summary_raw_hp2
summary_doc = nlp(summary)

In [ ]:
similarity_scores = [summary_doc.similarity(doc) for doc in dataset_docs]

most_similar_index = similarity_scores.index(max(similarity_scores))
most_similar_record = dataset[most_similar_index]


print("Given Synopsis is:", summary,'\n')
most_similar_record[:1000]

In [ ]:
df.columns

# NLTK
Preprocess your summary and dataset records using NLTK's tokenization, stemming, or other text processing functions.  
Represent the texts using appropriate numerical representations such as TF-IDF (Term Frequency-Inverse Document Frequency).  
Compute the similarity between the summary and each dataset record using measures like cosine similarity, Jaccard similarity, or other similarity metrics available in NLTK.  
Sort the records based on their similarity scores to find the most similar one.

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.metrics.distance import edit_distance
from nltk.metrics.distance import jaccard_distance


In [ ]:
# summary = "This is the summary of the text."
# dataset = ["Record 1", "Record 2", ..., "Record 100000"]

# Tokenize the summary
summary_tokens = word_tokenize(summary.lower())

# Remove stopwords from the summary
stop_words = set(stopwords.words("english"))
summary_filtered = [token for token in summary_tokens if token not in stop_words]

# Lemmatize the summary
lemmatizer = WordNetLemmatizer()
summary_lemmatized = [lemmatizer.lemmatize(token) for token in summary_filtered]

# Tokenize, remove stopwords, and lemmatize the dataset records
dataset_filtered = []
for record in dataset_clean_text:
    record_tokens = word_tokenize(record.lower())
    record_filtered = [token for token in record_tokens if token not in stop_words]
    record_lemmatized = [lemmatizer.lemmatize(token) for token in record_filtered]
    dataset_filtered.append(record_lemmatized)


In [ ]:
similarity_scores = []
for record in dataset_filtered:
    # Calculate the similarity using the Jaccard distance
    similarity = 1 - jaccard_distance(set(summary_lemmatized), set(record))
    similarity_scores.append(similarity)


In [ ]:
most_similar_index = similarity_scores.index(max(similarity_scores))
most_similar_record = dataset_clean_text[most_similar_index]


In [ ]:
most_similar_record

# GENSIM
Preprocess your summary and dataset records using tokenization, stemming, or other techniques available in gensim or NLTK.  
Train or load a Word2Vec model using gensim to generate word embeddings.  
Compute the sentence or document embeddings by averaging or pooling the word embeddings.  
Calculate the similarity between the summary and each dataset record using cosine similarity or other similarity measures available in gensim.  
Rank the records based on their similarity scores to identify the most similar one.

In [ ]:
import gensim
from gensim.models import Word2Vec, Doc2Vec
from gensim import corpora
from gensim.similarities import MatrixSimilarity


In [ ]:
# summary = "This is the summary of the text."
# dataset = ["Record 1", "Record 2", ..., "Record 100000"]

summary_raw_hp2 = "In this movie Ginny open it with a diary, and there is a big snake. Emma Watson as hermione, ron help Harry to defend from Voldemort, also Harry can talk to it in the girls bathroom. "
summary_raw_hp8 = "In this movie Harry have destroyed most Horcruxes, and he Hermione and Ron Have to fight Voldemort. There is a huge battle of wizards in the castle of Hogwarts. Weasly die in the fight, but one twin lose his ear. "
summary_raw_gf='a gangster try to get into a family business,Corleone with the italians. they are in the mafia family, the head of the Corleone Mafia family '
summary_raw_mi=" the IMF agent Ethan Hunt gets bretrayed and have to steal the noc list."
dataset_gensim = df['description'].tolist()
summary=summary_raw_hp8

# Tokenize the summary
summary_tokens = gensim.utils.simple_preprocess(summary.lower())

# Tokenize the dataset records
dataset_tokens = [gensim.utils.simple_preprocess(record.lower()) for record in dataset_gensim]


In [ ]:
# model = Word2Vec(dataset_tokens, min_count=1) #, size=100
model = Doc2Vec(word_tokenize(dataset_tokens[0]), min_count=1)


In [ ]:
summary_embedding = model.infer_vector(summary_tokens)
dataset_embeddings = [model.infer_vector(tokens) for tokens in dataset_tokens]


In [ ]:
corpus = [summary_embedding] + dataset_embeddings
index = MatrixSimilarity(corpus)

similarity_scores = index[summary_embedding]


In [ ]:
most_similar_index = similarity_scores.argmax()
most_similar_record = dataset_gensim[most_similar_index]


In [ ]:
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

# Assuming you have a list of documents, where each document is a list of words
documents = [['word1', 'word2', 'word3'], ['word4', 'word5', 'word6'], ['word7', 'word8', 'word9']]

# Prepare tagged documents for training
tagged_documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(documents)]

# Train the Doc2Vec model
model = Doc2Vec(tagged_documents, vector_size=100, epochs=10)

# Infer a vector for a new document
new_document = ['word10', 'word11', 'word12']
vector = model.infer_vector(new_document)

# Find the most similar documents
similar_documents = model.docvecs.most_similar(positive=[vector], topn=1)

# Retrieve the most similar document index
most_similar_index = similar_documents[0][0]

# Retrieve the most similar document from the list
most_similar_document = documents[most_similar_index]

print("Most similar document:", most_similar_document)


# Fuzzy Wuzzy
 FuzzyWuzzy is a string matching library in Python that provides various methods for fuzzy string comparison. While it may not be as comprehensive as using NLP libraries like spaCy or NLTK, it can still be useful for certain text similarity tasks. 
 Please note that FuzzyWuzzy primarily focuses on string matching based on character-level similarities and may not capture semantic meaning as effectively as NLP-based approaches. 
 Here's an example of how you can use FuzzyWuzzy for identifying similar movie synopses:

In [ ]:
pip install fuzzywuzzy


In [ ]:
from fuzzywuzzy import fuzz, process


In [ ]:
synopses = [
    "In a galaxy far, far away...",
    "A group of friends embark on an adventure...",
    "Two detectives investigate a series of crimes...",
    # Add more synopses here
]


In [ ]:
def find_most_similar_synopsis(query_synopsis, synopses_list):
    similarity_scores = []
    for synopsis in synopses_list:
        similarity = fuzz.token_set_ratio(query_synopsis, synopsis)
        similarity_scores.append(similarity)
    most_similar_index = similarity_scores.index(max(similarity_scores))
    most_similar_synopsis = synopses_list[most_similar_index]
    return most_similar_synopsis


In [ ]:
query = "A group of friends embark on a space journey..."
most_similar = find_most_similar_synopsis(query, synopses)
print(most_similar)
